In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly import figure_factory as ff
from ddm_model import DDM, Simulation, Threshold

# Base Model (Pure DDM)

In [2]:
th1 = Threshold(15, type="collapsing", collapse_arg=(15,0.005,8))

In [12]:
ddm_one = DDM(drift_rate=0.017348, noise_mag=0.5, threshold=th1,initial_condition=0.03) #drift rate max is around 0.07.

In [13]:
simulation_one = Simulation(ddm_one, 1, 1000,15)

In [14]:
simulation_one.plot_trajectories()

In [15]:
simulation_one.reaction_times.mean()

np.float64(425.46666666666664)

In [16]:
simulation_one.reaction_times

array([[333.],
       [430.],
       [656.],
       [197.],
       [383.],
       [278.],
       [240.],
       [818.],
       [746.],
       [570.],
       [532.],
       [167.],
       [125.],
       [120.],
       [787.]])

In [17]:
fig = ff.create_distplot([simulation_one.reaction_times.flatten()], ['Response Times'], show_hist=False, show_rug=False)

In [18]:
fig.show()

In [10]:
simulation_one.error_rate

0.0

In [11]:
simulation_one.simulated_results

[('A', 'A'),
 ('A', 'A'),
 ('A', 'A'),
 ('A', 'A'),
 ('B', 'B'),
 ('A', 'A'),
 ('B', 'B'),
 ('A', 'A'),
 ('A', 'A'),
 ('A', 'A'),
 ('B', 'B'),
 ('B', 'B'),
 ('B', 'B'),
 ('B', 'B'),
 ('A', 'A')]

Trajectory simulation data:

In [ ]:
pd.DataFrame(simulation_one.simulated_trajectories).T